# I. Importing Modules

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
import statsmodels.api as stsm
from statsmodels.formula.api import ols

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# II. Load Data

In [ ]:
df = pd.read_csv('/kaggle/input/playground-series-s3e22/train.csv')
df.drop('id',axis=1,inplace=True)
df

In [ ]:
# df = df.fillna(0)
df.shape

Based on Original dataset, Hospital Number refers to **the case number assigned to the horse (may not be unique if the horse is treated > 1 time)**
-> The data type should be String (object)

In [ ]:
df["hospital_number"] = df["hospital_number"].apply(np.str_)
df.dtypes

# III. Simple EDA

In [ ]:
y = np.array([(df.outcome  == 'lived').sum(), (df.outcome  == 'euthanized').sum(), (df.outcome  == 'died').sum()])
outcome_labels = ["Lived", "Euthanized", "Died"]

expl = (0.05, 0.05, 0.05)
plt.pie(y, labels = outcome_labels, explode=expl, autopct='%1.0f%%', shadow=True)
plt.show() 

In [ ]:
string_cols = df.select_dtypes(include=['object'])
string_column_list = [col for col in string_cols.columns]

encode = LabelEncoder()
for column in string_column_list:
    df[column] = encode.fit_transform(df[column].values)

In [ ]:
df

In [ ]:
# # Fit the ANOVA model
# model = ols('outcome ~ surgery * age * hospital_number * temp_of_extremities', data=df).fit()

# # Perform ANOVA
# anova_table = stsvm.stats.anova_lm(model, typ=2)
# anova_table 

## Oversampling using SMOTE
SMOTE is an abbreviation for "Synthetic Minority Over-sampling Technique". It is a method used in data analysis and machine learning, particularly in the context of classification, to address the problem of class imbalance. The class imbalance problem occurs when the number of samples in the dominant class is significantly greater than the number of samples in the minority class.

In [ ]:
x = df.drop(['outcome'], axis=1)
y = df['outcome']

In [ ]:
sm = SMOTE(random_state=30)
x_sampling, y_sampling = sm.fit_resample(x, y)

In [ ]:
print(x_sampling.shape)
print(y_sampling.shape)

In [ ]:
y_arr = np.array([(y_sampling == 2).sum(), (y_sampling == 1).sum(), (y_sampling == 0).sum()])
outcome_labels = ["Lived", "Euthanized", "Died"]

expl = (0.05, 0.05, 0.05)
plt.pie(y_arr, labels = outcome_labels, explode=expl, autopct='%1.0f%%', shadow=True)
plt.show() 

In [ ]:
float_cols = df.select_dtypes(include=['float64'])
column_list = [col for col in float_cols.columns]

In [ ]:
num_cols = len(column_list)
num_rows = 1
fig, axes = plt.subplots(num_rows, num_cols, figsize=(16, 4))

for i, kolom in enumerate(column_list):
    if num_rows == 1:
        ax = axes[i]
    else:
        row = i // num_cols
        col = i % num_cols
        ax = axes[row, col]
    
    ax.boxplot(df[kolom])
    ax.set_title(kolom)

plt.tight_layout()
plt.show()

# NAIVE BAYES

## Without SMOTE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

naive_bayes_model = GaussianNB()
naive_bayes_model.fit(X_train, y_train)
y_pred = naive_bayes_model.predict(X_test)


print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

## With SMOTE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_sampling, y_sampling, test_size=0.3, random_state=42)

naive_bayes_model = GaussianNB()
naive_bayes_model.fit(X_train, y_train)
y_pred = naive_bayes_model.predict(X_test)


print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Neural Network